In [1]:
ds_dataset = spark.sql("select * from default.db_dettagli_prodotti_clean_4_csv where tecnico is not null").dropDuplicates(['id_prodotto'])

In [2]:
ds_dataset.createOrReplaceTempView("ds_dataset")

In [3]:
%sql

select tecnico, count(*) as num from ds_dataset group by tecnico

tecnico,num
si,881
no,486


In [4]:
#divido i dati mappati manualmente in test e training randomicamente
train, test = ds_dataset.randomSplit([0.9, 0.1], seed=12345)

In [5]:
#Attraverso le regular expression pulisco le descrizioni rimuovendo numeri e mettendo il testo in minuscolo
train.createOrReplaceTempView("train")
train_cleaned= spark.sql("select regexp_replace(descrizione, '[0-9]', ' ')as description_cleaned, tecnico as tecnico from train")
display(train_cleaned.select("*").limit(10))

description_cleaned,tecnico
"abbiamo sviluppato questa giacca da uomo per proteggerti dalla pioggia e dal vento durante le uscite di running inferiori ad un'ora, ad andatura moderata.questa giacca ti proteggerà dalla pioggia e dal vento durante le sessioni di running inferiori ad un'ora grazie al componente impermeabile! due tasche con zip.i vantaggi antivento tessuto principale antivento. impermeabilità protezione parziale dalla pioggia: tessuto principale impermeabile. cuciture no. traspirabilità due piccole aperture sotto le braccia per facilitare la circolazione dell'aria. tasche due tasche con zip per mettere le chiavi, i guanti.informazioni tecniche protezione parziale dalla pioggia adatta per il running questa giacca ti proteggerà dalla pioggia e dal vento grazie al componente con rivestimento impermeabile. le cuciture non sono impermeabilizzate. questo significa che se la pioggia dura a lungo l'acqua puà² passare dalle cuciture. questo non è un problema per le uscite di running inferiori ad un'ora. ãƒë† perà² importante sapere che questo tipo di design non è adatto per proteggere integralmente dalla pioggia durante le escursioni o i trail lunghi.",no
"i nostri appassionati di atletica hanno sviluppato questo prodotto per tenerti al caldo durante il riscaldamento, negli allenamenti e nelle gare.tieni i muscoli al caldo durante il riscaldamento grazie alla qualità termica del componente e toglili semplicemente senza dover togliere le scarpe grazie alle zip laterali specifiche.i vantaggi calore componente studiato appositamente per favorire il riscaldamento muscolare. facile da indossare / togliere zip laterali dalle ginocchia ai piedi per sfilarli senza togliere le scarpe. tasche due tasche laterali con zip per riporre i propri effetti personali.informazioni tecniche pantaloni facili da mettere e da togliere la lunghezza della cerniera permette di aprirli fino a sotto al ginocchio. in questo modo si possono mettere e togliere i pantaloni senza dover togliere le scarpe da atletica, anche se sono scarpe con chiodi. grazie alla linguetta autobloccante, la cerniera rimane bene a posto e non si muove più durate l'attività .",si
"abbiamo ideato questa canotta per le donne che corrono con tempo caldo.il tessuto fluido, il taglio ampio e il retro vogatore di questa canotta da running ti conquisteranno! un comfort ed una libertà di movimento incredibili!i vantaggi traspirazione tessuto traspirante per rimanere all'asciutto. fluidità tessuto morbido e fluido che ti conquisterà ! libertà di movimento taglio ampio per maggiore libertà di movimento.informazioni tecniche perchãƒâ€° scegliere una t-shirt specifica per correre? il sudore è una reazione naturale che avviene quando fa caldo e quando si fa uno sforzo fisico. ãƒë† meglio non indossare una t-shirt in cotone, perché il tessuto assorbe il sudore ma raffredda rapidamente la pelle perché non disperde il sudore verso l'esterno e non asciuga rapidamente. scegliendo una t-shirt adatta si aumenta il comfort, qualunque sia la durata della pratica. traspirabilitãƒâ‚¬ il componente utilizzato limita la sensazione di umido grazie all'assorbimento, alla dispersione e all'asciugatura del sudore prodotto dal corpo in attività . fluiditãƒâ‚¬ taglio un po' ampio e componente molto fluido per la massima libertà di movimento. il % delle collaudatrici ha apprezzato questa canotta per il suo materiale morbido e fluido! tocca a te provarla!",no
"Progettate per la corsa su lunghe distanze, queste scarpe da donna sono caratterizzate da una tomaia in mesh traspirante con zone di rinforzo per un supporto mirato. La tecnologia Torsion System assicura una falcata fluida, "" mentre lintersuola è dotata della nostra ammortizzazione più reattiva per restituirti energia a ogni passo"", Calzata regolare, Chiusura con lacci, ""Tomaia in air mesh tecnologia Tailored Fibre Placement per un sostegno mirato sullarco plantare"", Fodera in tessuto suola Stretchweb con gomma Continental™ per un grip eccezionale, ma

In [6]:
#creazione token, tramite regex gli dico dove splittare il testo
from pyspark.ml.feature import RegexTokenizer
from pyspark.sql.functions import col, udf
from pyspark.sql.types import IntegerType

regexTokenizer = RegexTokenizer(inputCol="description_cleaned", outputCol="words", pattern="\\s|\\p{P}")

# check our results
countTokens = udf(lambda words: len(words), IntegerType())
tokenized = regexTokenizer.transform(train_cleaned)
tokenized_counts = tokenized.select("description_cleaned", "words").withColumn("tokens", countTokens(col("words")))
display(tokenized_counts.select("*").limit(10))

description_cleaned,words,tokens
"abbiamo sviluppato questa giacca da uomo per proteggerti dalla pioggia e dal vento durante le uscite di running inferiori ad un'ora, ad andatura moderata.questa giacca ti proteggerà dalla pioggia e dal vento durante le sessioni di running inferiori ad un'ora grazie al componente impermeabile! due tasche con zip.i vantaggi antivento tessuto principale antivento. impermeabilità protezione parziale dalla pioggia: tessuto principale impermeabile. cuciture no. traspirabilità due piccole aperture sotto le braccia per facilitare la circolazione dell'aria. tasche due tasche con zip per mettere le chiavi, i guanti.informazioni tecniche protezione parziale dalla pioggia adatta per il running questa giacca ti proteggerà dalla pioggia e dal vento grazie al componente con rivestimento impermeabile. le cuciture non sono impermeabilizzate. questo significa che se la pioggia dura a lungo l'acqua puà² passare dalle cuciture. questo non è un problema per le uscite di running inferiori ad un'ora. ãƒë† perà² importante sapere che questo tipo di design non è adatto per proteggere integralmente dalla pioggia durante le escursioni o i trail lunghi.","List(abbiamo, sviluppato, questa, giacca, da, uomo, per, proteggerti, dalla, pioggia, e, dal, vento, durante, le, uscite, di, running, inferiori, ad, un, ora, ad, andatura, moderata, questa, giacca, ti, proteggerà , dalla, pioggia, e, dal, vento, durante, le, sessioni, di, running, inferiori, ad, un, ora, grazie, al, componente, impermeabile, due, tasche, con, zip, i, vantaggi, antivento, tessuto, principale, antivento, impermeabilità , protezione, parziale, dalla, pioggia, tessuto, principale, impermeabile, cuciture, no, traspirabilità , due, piccole, aperture, sotto, le, braccia, per, facilitare, la, circolazione, dell, aria, tasche, due, tasche, con, zip, per, mettere, le, chiavi, i, guanti, informazioni, tecniche, protezione, parziale, dalla, pioggia, adatta, per, il, running, questa, giacca, ti, proteggerà , dalla, pioggia, e, dal, vento, grazie, al, componente, con, rivestimento, impermeabile, le, cuciture, non, sono, impermeabilizzate, questo, significa, che, se, la, pioggia, dura, a, lungo, l, acqua, puà², passare, dalle, cuciture, questo, non, è, un, problema, per, le, uscite, di, running, inferiori, ad, un, ora, ãƒë, perà², importante, sapere, che, questo, tipo, di, design, non, è, adatto, per, proteggere, integralmente, dalla, pioggia, durante, le, escursioni, o, i, trail, lunghi)",174
"i nostri appassionati di atletica hanno sviluppato questo prodotto per tenerti al caldo durante il riscaldamento, negli allenamenti e nelle gare.tieni i muscoli al caldo durante il riscaldamento grazie alla qualità termica del componente e toglili semplicemente senza dover togliere le scarpe grazie alle zip laterali specifiche.i vantaggi calore componente studiato appositamente per favorire il riscaldamento muscolare. facile da indossare / togliere zip laterali dalle ginocchia ai piedi per sfilarli senza togliere le scarpe. tasche due tasche laterali con zip per riporre i propri effetti personali.informazioni tecniche pantaloni facili da mettere e da togliere la lunghezza della cerniera permette di aprirli fino a sotto al ginocchio. in questo modo si possono mettere e togliere i pantaloni senza dover togliere le scarpe da atletica, anche se sono scarpe con chiodi. grazie alla linguetta autobloccante, la cerniera rimane bene a posto e non si muove più durate l'attività .","List(i, nostri, appassionati, di, atletica, hanno, sviluppato, questo, prodotto, per, tenerti, al, caldo, durante, il, riscaldamento, negli, allenamenti, e, nelle, gare, tieni, i, muscoli, al, caldo, durante, il, riscaldamento, grazie, alla, qualità , termica, del, componente, e, toglili, semplicemente, senza, dover, togliere, le, scarpe, grazie, alle, zip, laterali, specifiche, i, vantaggi, calore, componente, studiato, appositamente, per, favorire, il, riscaldamento, muscolare, facile, da, indossare, toglie

In [7]:
#esplodo l'elenco di token e conto la frequenza dei singoli termini
from pyspark.sql.functions import explode, desc
tokens = tokenized.select(explode(col("words")).alias("word")).groupBy(col("word")).count().orderBy(desc("count"))
display(tokens.select("*").limit(20))

word,count
e,5023
per,4561
di,4241
in,2635
la,2508
con,2282
il,2275
una,1936
a,1936
un,1700


In [8]:
from pyspark.ml.feature import StopWordsRemover
remover = StopWordsRemover(inputCol="words", outputCol="cleaned", caseSensitive=False)

In [9]:
#rimuovo il vocabolario italiano di stopwords
remover.setStopWords(StopWordsRemover.loadDefaultStopWords("italian"))
cleaned = remover.transform(tokenized)
cleaned.show()

+--------------------+-------+--------------------+--------------------+
 description_cleaned|tecnico| words| cleaned|
+--------------------+-------+--------------------+--------------------+
abbiamo sviluppat...| no|[abbiamo, svilupp...|[sviluppato, giac...|
i nostri appassio...| si|[i, nostri, appas...|[appassionati, at...|
abbiamo ideato qu...| no|[abbiamo, ideato,...|[ideato, canotta,...|
Progettate per la...| si|[progettate, per,...|[progettate, cors...|
Il padre degli sh...| si|[il, padre, degli...|[padre, short, st...|
La scarpa da cors...| si|[la, scarpa, da, ...|[scarpa, corsa, g...|
Giacca a vento tr...| si|[giacca, a, vento...|[giacca, vento, t...|
le nostre equipe ...| no|[le, nostre, equi...|[equipe, sviluppa...|
Non puoi decidere...| si|[non, puoi, decid...|[puoi, decidere, ...|
Calzatura in memb...| si|[calzatura, in, m...|[calzatura, membr...|
La maglia Nike Ic...| si|[la, maglia, nike...|[maglia, nike, ic...|
T-shirt running d...| si|[t, shirt, runnin...|[t, shirt, runnin...|
Scarpe running ne...| si|[scarpe, running,...|[scarpe, running,...|
abbiamo ideato la...| no|[abbiamo, ideato,...|[ideato, t, shirt...|
Scarpa running do...| si|[scarpa, running,...|[scarpa, running,...|
abbiamo ideato qu...| no|[abbiamo, ideato,...|[ideato, pantalon...|
la nostra equipe ...| si|[la, nostra, equi...|[equipe, appassio...|
La maglietta Runn...| no|[la, maglietta, r...|[maglietta, runni...|
La maglia da runn...| no|[la, maglia, da, ...|[maglia, running,...|
Runner Tank di La...| si|[runner, tank, di...|[runner, tank, sp...|
+--------------------+-------+--------------------+--------------------+
only showing top 20 rows

In [10]:
tokens = cleaned.select(explode(col("cleaned")).alias("word")).groupBy(col("word")).count().orderBy(desc("count"))
display(tokens.select("*").limit(20))

word,count
running,1588
tessuto,1106
tipologia,975
tecnologia,944
utilizzo,784
traspirante,769
comfort,764
fit,555
grazie,520
scarpe,510


In [11]:
#dopo aver notato che ci sono ancora parole poco significative filtro ulteriormente tenendo solo le parole di lunghezza maggiore a 2
from pyspark.sql.functions import udf
from pyspark.sql.types import ArrayType, StringType


def filter_by_len(words):
  filtered = [word for word in words if len(word) > 2]
  return filtered


filter_by_len_udf = udf(filter_by_len, ArrayType(StringType()))

In [12]:
filtered = cleaned.withColumn("filtered", filter_by_len_udf(col("cleaned")))
filtered.show()

+--------------------+-------+--------------------+--------------------+--------------------+
 description_cleaned|tecnico| words| cleaned| filtered|
+--------------------+-------+--------------------+--------------------+--------------------+
abbiamo sviluppat...| no|[abbiamo, svilupp...|[sviluppato, giac...|[sviluppato, giac...|
i nostri appassio...| si|[i, nostri, appas...|[appassionati, at...|[appassionati, at...|
abbiamo ideato qu...| no|[abbiamo, ideato,...|[ideato, canotta,...|[ideato, canotta,...|
Progettate per la...| si|[progettate, per,...|[progettate, cors...|[progettate, cors...|
Il padre degli sh...| si|[il, padre, degli...|[padre, short, st...|[padre, short, st...|
La scarpa da cors...| si|[la, scarpa, da, ...|[scarpa, corsa, g...|[scarpa, corsa, g...|
Giacca a vento tr...| si|[giacca, a, vento...|[giacca, vento, t...|[giacca, vento, t...|
le nostre equipe ...| no|[le, nostre, equi...|[equipe, sviluppa...|[equipe, sviluppa...|
Non puoi decidere...| si|[non, puoi, decid...|[puoi, decidere, ...|[puoi, decidere, ...|
Calzatura in memb...| si|[calzatura, in, m...|[calzatura, membr...|[calzatura, membr...|
La maglia Nike Ic...| si|[la, maglia, nike...|[maglia, nike, ic...|[maglia, nike, ic...|
T-shirt running d...| si|[t, shirt, runnin...|[t, shirt, runnin...|[shirt, running, ...|
Scarpe running ne...| si|[scarpe, running,...|[scarpe, running,...|[scarpe, running,...|
abbiamo ideato la...| no|[abbiamo, ideato,...|[ideato, t, shirt...|[ideato, shirt, r...|
Scarpa running do...| si|[scarpa, running,...|[scarpa, running,...|[scarpa, running,...|
abbiamo ideato qu...| no|[abbiamo, ideato,...|[ideato, pantalon...|[ideato, pantalon...|
la nostra equipe ...| si|[la, nostra, equi...|[equipe, appassio...|[equipe, appassio...|
La maglietta Runn...| no|[la, maglietta, r...|[maglietta, runni...|[maglietta, runni...|
La maglia da runn...| no|[la, maglia, da, ...|[maglia, running,...|[maglia, running,...|
Runner Tank di La...| si|[runner, tank, di...|[runner, tank, sp...|[runner, tank, sp...|
+--------------------+-------+--------------------+--------------------+--------------------+
only showing top 20 rows

In [13]:
tokens = filtered.select(explode(col("filtered")).alias("word")).groupBy(col("word")).count().orderBy(desc("count"))
display(tokens.select("*").limit(200))

word,count
running,1588
tessuto,1106
tipologia,975
tecnologia,944
utilizzo,784
traspirante,769
comfort,764
fit,555
grazie,520
scarpe,510


In [14]:
#aggiungo una colonna per ngrammi di due lettere e una per ngrammi di 3 lettere
from pyspark.ml.feature import NGram
ngrams2 = NGram(n=2, inputCol="filtered", outputCol="ngrams_2")
ngrams = ngrams2.transform(filtered)
ngrams3 = NGram(n=3, inputCol="filtered", outputCol="ngrams_3")
ngrams = ngrams3.transform(ngrams)

ngrams.show()

+--------------------+-------+--------------------+--------------------+--------------------+--------------------+--------------------+
 description_cleaned|tecnico| words| cleaned| filtered| ngrams_2| ngrams_3|
+--------------------+-------+--------------------+--------------------+--------------------+--------------------+--------------------+
abbiamo sviluppat...| no|[abbiamo, svilupp...|[sviluppato, giac...|[sviluppato, giac...|[sviluppato giacc...|[sviluppato giacc...|
i nostri appassio...| si|[i, nostri, appas...|[appassionati, at...|[appassionati, at...|[appassionati atl...|[appassionati atl...|
abbiamo ideato qu...| no|[abbiamo, ideato,...|[ideato, canotta,...|[ideato, canotta,...|[ideato canotta, ...|[ideato canotta d...|
Progettate per la...| si|[progettate, per,...|[progettate, cors...|[progettate, cors...|[progettate corsa...|[progettate corsa...|
Il padre degli sh...| si|[il, padre, degli...|[padre, short, st...|[padre, short, st...|[padre short, sho...|[padre short stud...|
La scarpa da cors...| si|[la, scarpa, da, ...|[scarpa, corsa, g...|[scarpa, corsa, g...|[scarpa corsa, co...|[scarpa corsa gel...|
Giacca a vento tr...| si|[giacca, a, vento...|[giacca, vento, t...|[giacca, vento, t...|[giacca vento, ve...|[giacca vento tra...|
le nostre equipe ...| no|[le, nostre, equi...|[equipe, sviluppa...|[equipe, sviluppa...|[equipe sviluppat...|[equipe sviluppat...|
Non puoi decidere...| si|[non, puoi, decid...|[puoi, decidere, ...|[puoi, decidere, ...|[puoi decidere, d...|[puoi decidere te...|
Calzatura in memb...| si|[calzatura, in, m...|[calzatura, membr...|[calzatura, membr...|[calzatura membra...|[calzatura membra...|
La maglia Nike Ic...| si|[la, maglia, nike...|[maglia, nike, ic...|[maglia, nike, ic...|[maglia nike, nik...|[maglia nike icon...|
T-shirt running d...| si|[t, shirt, runnin...|[t, shirt, runnin...|[shirt, running, ...|[shirt running, r...|[shirt running uo...|
Scarpe running ne...| si|[scarpe, running,...|[scarpe, running,...|[scarpe, running,...|[scarpe running, ...|[scarpe running n...|
abbiamo ideato la...| no|[abbiamo, ideato,...|[ideato, t, shirt...|[ideato, shirt, r...|[ideato shirt, sh...|[ideato shirt run...|
Scarpa running do...| si|[scarpa, running,...|[scarpa, running,...|[scarpa, running,...|[scarpa running, ...|[scarpa running d...|
abbiamo ideato qu...| no|[abbiamo, ideato,...|[ideato, pantalon...|[ideato, pantalon...|[ideato pantalonc...|[ideato pantalonc...|
la nostra equipe ...| si|[la, nostra, equi...|[equipe, appassio...|[equipe, appassio...|[equipe appassion...|[equipe appassion...|
La maglietta Runn...| no|[la, maglietta, r...|[maglietta, runni...|[maglietta, runni...|[maglietta runnin...|[maglietta runnin...|
La maglia da runn...| no|[la, maglia, da, ...|[maglia, running,...|[maglia, running,...|[maglia running, ...|[maglia running n...|
Runner Tank di La...| si|[runner, tank, di...|[runner, tank, sp...|[runner, tank, sp...|[runner tank, tan...|[runner tank spor...|
+--------------------+-------+--------------------+--------------------+--------------------+--------------------+--------------------+
only showing top 20 rows

In [15]:
#creo una colonna contenente i singoli termini e gli ngrammi
def union_ngrams(c1,c2,c3):
  return c1 + c2 +c3

union_ngrams_udf = udf(union_ngrams, ArrayType(StringType()))

ngrams_final = ngrams.filter("filtered is not Null").withColumn("ngrams", union_ngrams_udf(col("filtered"), col("ngrams_2"),col("ngrams_3")))
ngrams_final.show()

+--------------------+-------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
 description_cleaned|tecnico| words| cleaned| filtered| ngrams_2| ngrams_3| ngrams|
+--------------------+-------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
abbiamo sviluppat...| no|[abbiamo, svilupp...|[sviluppato, giac...|[sviluppato, giac...|[sviluppato giacc...|[sviluppato giacc...|[sviluppato, giac...|
i nostri appassio...| si|[i, nostri, appas...|[appassionati, at...|[appassionati, at...|[appassionati atl...|[appassionati atl...|[appassionati, at...|
abbiamo ideato qu...| no|[abbiamo, ideato,...|[ideato, canotta,...|[ideato, canotta,...|[ideato canotta, ...|[ideato canotta d...|[ideato, canotta,...|
Progettate per la...| si|[progettate, per,...|[progettate, cors...|[progettate, cors...|[progettate corsa...|[progettate corsa...|[progettate, cors...|
Il padre degli sh...| si|[il, padre, degli...|[padre, short, st...|[padre, short, st...|[padre short, sho...|[padre short stud...|[padre, short, st...|
La scarpa da cors...| si|[la, scarpa, da, ...|[scarpa, corsa, g...|[scarpa, corsa, g...|[scarpa corsa, co...|[scarpa corsa gel...|[scarpa, corsa, g...|
Giacca a vento tr...| si|[giacca, a, vento...|[giacca, vento, t...|[giacca, vento, t...|[giacca vento, ve...|[giacca vento tra...|[giacca, vento, t...|
le nostre equipe ...| no|[le, nostre, equi...|[equipe, sviluppa...|[equipe, sviluppa...|[equipe sviluppat...|[equipe sviluppat...|[equipe, sviluppa...|
Non puoi decidere...| si|[non, puoi, decid...|[puoi, decidere, ...|[puoi, decidere, ...|[puoi decidere, d...|[puoi decidere te...|[puoi, decidere, ...|
Calzatura in memb...| si|[calzatura, in, m...|[calzatura, membr...|[calzatura, membr...|[calzatura membra...|[calzatura membra...|[calzatura, membr...|
La maglia Nike Ic...| si|[la, maglia, nike...|[maglia, nike, ic...|[maglia, nike, ic...|[maglia nike, nik...|[maglia nike icon...|[maglia, nike, ic...|
T-shirt running d...| si|[t, shirt, runnin...|[t, shirt, runnin...|[shirt, running, ...|[shirt running, r...|[shirt running uo...|[shirt, running, ...|
Scarpe running ne...| si|[scarpe, running,...|[scarpe, running,...|[scarpe, running,...|[scarpe running, ...|[scarpe running n...|[scarpe, running,...|
abbiamo ideato la...| no|[abbiamo, ideato,...|[ideato, t, shirt...|[ideato, shirt, r...|[ideato shirt, sh...|[ideato shirt run...|[ideato, shirt, r...|
Scarpa running do...| si|[scarpa, running,...|[scarpa, running,...|[scarpa, running,...|[scarpa running, ...|[scarpa running d...|[scarpa, running,...|
abbiamo ideato qu...| no|[abbiamo, ideato,...|[ideato, pantalon...|[ideato, pantalon...|[ideato pantalonc...|[ideato pantalonc...|[ideato, pantalon...|
la nostra equipe ...| si|[la, nostra, equi...|[equipe, appassio...|[equipe, appassio...|[equipe appassion...|[equipe appassion...|[equipe, appassio...|
La maglietta Runn...| no|[la, maglietta, r...|[maglietta, runni...|[maglietta, runni...|[maglietta runnin...|[maglietta runnin...|[maglietta, runni...|
La maglia da runn...| no|[la, maglia, da, ...|[maglia, running,...|[maglia, running,...|[maglia running, ...|[maglia running n...|[maglia, running,...|
Runner Tank di La...| si|[runner, tank, di...|[runner, tank, sp...|[runner, tank, sp...|[runner tank, tan...|[runner tank spor...|[runner, tank, sp...|
+--------------------+-------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
only showing top 20 rows

In [16]:
##genero e assegno gli hash e i pesi
from pyspark.ml.feature import HashingTF
from pyspark.ml.feature import IDF

hashing_tf = HashingTF(inputCol="ngrams", outputCol="rawFeatures")
idf = IDF(inputCol="rawFeatures", outputCol="features")
hash_dataset = hashing_tf.transform(ngrams_final)
idf_model = idf.fit(hash_dataset)
idf_dataset = idf_model.transform(hash_dataset)

In [17]:
#aggiungo la colonna label che assegna un valore alle categorie e imposto il metodo di decodifica
from pyspark.ml.feature import IndexToString, StringIndexer

indexer = StringIndexer(inputCol="tecnico", outputCol="label")
indexer_model = indexer.fit(idf_dataset)
indexed = indexer_model.transform(idf_dataset)
converter = IndexToString(inputCol="prediction", outputCol="prediction_tecnico", labels=indexer_model.labels)

In [18]:
#imposto NaiveBayes
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit

nb = NaiveBayes(smoothing=1.0, modelType="multinomial", labelCol="label", featuresCol="features")
paramGrid = ParamGridBuilder()\
    .addGrid(nb.smoothing, [0.1, 0.5, 1.0]) \
    .build()

tvs = TrainValidationSplit(estimator=nb,
                           estimatorParamMaps=paramGrid,
                           evaluator=MulticlassClassificationEvaluator(),
                           trainRatio=0.8)

In [19]:
#Applico Naive-Bayes
nb_model = tvs.fit(indexed)

/databricks/spark/python/pyspark/ml/util.py:737: UserWarning: Cannot find mlflow module. To enable MLflow logging, install mlflow from PyPI.
 warnings.warn(_MLflowInstrumentation._NO_MLFLOW_WARNING)

In [20]:
#Svolgo gli stessi passaggi sul set di test
test.createOrReplaceTempView("test")
test_cleaned = spark.sql("select id_prodotto, sito, nome, Descrizione, regexp_replace(descrizione,'[0-9]', ' ') as description_cleaned, tecnico as tecnico from test")
test_tokenized = regexTokenizer.transform(test_cleaned)
test_cleaned = remover.transform(test_tokenized)
test_filtered = test_cleaned.withColumn("filtered", filter_by_len_udf(col("cleaned")))
test_ngrams = ngrams2.transform(test_filtered)
test_ngrams = ngrams3.transform(test_ngrams)
test_ngrams_final = test_ngrams.filter("filtered is not Null").withColumn("ngrams", union_ngrams_udf(col("filtered"), col("ngrams_2"), col("ngrams_3")))

test_hash_dataset = hashing_tf.transform(test_ngrams_final)
test_idf_dataset = idf_model.transform(test_hash_dataset)
test_indexed = indexer_model.transform(test_idf_dataset)

In [21]:
predictions = nb_model.transform(test_indexed).select("nome","id_prodotto", "sito", "descrizione", "label", "prediction", "tecnico")
predictions_decoded = converter.transform(predictions)
display(predictions_decoded.select("*").limit(20))

nome,id_prodotto,sito,descrizione,label,prediction,tecnico,prediction_tecnico
Energy On UW - maglietta tecnica - uomo,10862928,sportler,"Maglietta tecnica da uomo Energy On UW Shirt di Uyn molto leggera e confortevole, ideale per le attività sportive ad abbondante sudorazione, espelle rapidamente il calore per una performance sempre di alto livello. tessuto funzionale Natex di derivazione naturale batteriostatico e anti-odore taglio ergonomico e senza cuciture grazie alla tecnologia Hypermotion fori di diametro differente in aree specifiche microcoste e uso combinato di polipropilene per una traspirazione efficace girocollo maniche raglan per una maggiore mobilità bordo manica e fondo elastici loghi immagliati vestibilità Regular Fit",0.0,0.0,si,si
Core Light - pantaloncini running - uomo,10754648,sportler,"I pantaloncini running da uomo Odlo Shorts Core Light sono dinamici, sportivi ed estremamente leggeri. Sono dotati di una tasca per le chiavi e garantiscono una vestibilità comoda. tasca per chiave leggeri con una vestibilità comoda vita elastisca regolabile inserti in mesh per un comfort ottimale",1.0,1.0,no,no
Cascadia 14 - scarpe trail running - uomo,10833092,sportler,"Scarpe trail running da uomo Cascadia 14, versione migliorata della nota linea da trail di Brooks, sono perfette per tutti i runners che amano il brivido dell'off-road e vogliono godere di pestazione e grip ottimali anche sui terreni più difficili. La nuova tomaia protettiva, con inserti in Cordura® strategicamente posizionati nei punti di maggiore sollecitazione, offre comfort ideale anche sulle lunghe distanze. La suola TrailTrack introduce un nuovo disegno nei tasselli che favoriscono una presa sicura su fondi fangosi e friabili. tomaia in maglia a fibra robusta e traspirante inserti protettivi in Cordura® antiabrasione tecnologia BioMoGO DNA per ammortizzazione efficace intersuola con componenti PIVOT ridisegnati e ridimensionati per offrire stabilità ancora più elevata zona avampiede più spaziosa garantisce maggior comfort nella calzata, soprattutto sulle lunghe distanze suola TrailTrack per trazione sicura anche su fondi instabili e scivolosi tasselli con nuovo disegno dal grip ottimizzato a prova di fango e terreni friabili",0.0,0.0,si,si
t-shirt trail uomo grigio scuro-nero,8563596,decathlon,"sviluppata dalla nostra equipe composta da praticanti di trail, questa t-shirt maniche corte leggermente avvitata ti seguirà nei tuoi trail brevi e lunghi.ideata su misura per il trail, questa t-shirt resiste agli sfregamenti dello zaino. le zone in mesh permettono di avere traspirabilità e la mezza zip permette di controllare la ventilazione.i vantaggi resistenza all'abrasione il componente resiste agli sfregamenti dello zaino e protegge la t-shirt. ventilazione buona circolazione dell'aria grazie alle 3 zone in mesh e alla zip centrale. tasche due pratiche tasche a rete sulla schiena per la nutrizione e i rifiuti! traspirazione resta all'asciutto durante tutto il trail grazie al componente tecnico.informazioni tecniche dettagli dedicati al comfort dei praticanti di trail sviluppata per resistere agli sfregamenti, questa t-shirt ha anche un colletto che protegge da eventuali irritazioni provocate dallo zaino e dai colpi di sole! la ventilazione? sei tu a gestirla! la mezza zip di 24 cm permette di coprire tutto il collo durante le partenze dei trail, spesso al fresco. dopo qualche ora di corsa, potrai aprire la zip per fare circolare l'aria come preferisci! due tasche posizionate strategicamente le due tasche a rete di facile accesso e chiuse con una patta permettono di riporre i gel e i fazzolettini. e, secondo i nostri studi, la posizione sulla schiena è l'deale per l'equilibrio. vedi altri",1.0,1.0,no,no
scarpe uomo speedgoat 4,98386887,df sport specialist,"Speedgoat 4, che deve il nome all'atleta HOKA Karl â€œSpeedgoatâ€\x9d Meltzer, fa parte della premiata famiglia di prodotti nota per la facilità con cui affronta i terreni tecnici. Questa quarta edizion

In [22]:
#calcolo i parametri per vedere quanto funziona l'algoritmo
evaluator_precision = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",
                                              metricName="weightedPrecision")
precision = evaluator_precision.evaluate(predictions)
print("Test set weighted precision = " + str(precision))


evaluator_recall = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",
                                              metricName="weightedRecall")
recall = evaluator_recall.evaluate(predictions)
print("Test set weighted recall = " + str(recall))


evaluator_f1 = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction",
                                              metricName="f1")
f1 = evaluator_f1.evaluate(predictions)
print("Test set f1 = " + str(f1))

Test set weighted precision = 0.8686759131062527
Test set weighted recall = 0.8686131386861313
Test set f1 = 0.8668306406836126

In [23]:
display(predictions_decoded.filter("label <> prediction").select("*").limit(20))

nome,id_prodotto,sito,descrizione,label,prediction,tecnico,prediction_tecnico
scarpe uomo wave prodigy 2,98387863,df sport specialist,"Ottimizzata per la tua corsa. Reattiva, con una superba ammortizzazione grazie dalla tecnologia Parallel Wave di Mizuno e una calzata migliorata grazie al supporto strategicamente posizionato., PESO (G): 305, UTILIZZO:, Running, TIPOLOGIA:, Scarpe, UTILIZZO:, Asfalto, TIPOLOGIA:, Neutre, TIPOLOGIA:, Calzature, CF: J1GR1810-09"", no, da valutare",1.0,0.0,no,si
Nike Wmns Icnclsh Top Ss,367593,nencini,"TOP TRASPIRANTE CON DETTAGLI ISPIRATI ALLA TRADIZIONE. Realizzato in tessuto traspirante con tecnologia Dri-FIT, il top Nike Icon Clash offre un comfort leggero su qualsiasi distanza. CIRCOLAZIONE DELL'ARIA IDEALE Tessuto Nike Breathe per pelle fresca e asciutta. Inserto anteriore e posteriore in mesh per lasciar circolare l'aria fresca e fuoriuscire il calore. CONTINUA A LEGGERE",1.0,0.0,no,si
short uomo pro 365,98379771,df sport specialist,"Gli shorts Nike Pro ti aiutano a rendere ogni allenamento più facile. Sono dotati di una cintura Nike Pro a vista con cuciture lisce e piatte e una tasca a punta per offrirti comfort e sostegno per tutto il giorno., La tecnologia Dri-FIT allontana il sudore dalla pelle per favorire una rapida evaporazione e garantire pelle asciutta e comfort., Massime prestazioni, Cuciture piatte e lisce e compressive fit che scolpisce il tuo corpo e offre una vestibilità sicura e senza distrazioni, per uno strato base ideale., Comfort e sostegno superiori, Rivisitati con una cintura leggermente più alta, traspirante e più spessa per offrire maggiore sostegno., UTILIZZO, Running, UTILIZZO, Fitness, TIPOLOGIA, Boxer e slip, TIPOLOGIA, Intimo, CF BV5635-010",1.0,0.0,no,si
canotta trail uomo perf verde-giallo,8563603,decathlon,"immaginata dalle nostre equipe di praticanti di trail, questa canotta sciancrata è l'alleato delle tue performance nei trail di breve e lunga distanza!fatta su misura per il trail e grazie al componente principale, questa canotta resiste agli sfregamenti dello zaino. la mezza zip permette di gestire la ventilazione!i vantaggi resistenza all'abrasione il componente resiste agli sfregamenti dello zaino e protegge la canotta. traspirazione resta all'asciutto durante tutto il trail grazie al componente tecnico. leggerezza con 116g la taglia l, questa canotta è una vera e propria seconda pelle! ventilazione buona circolazione dell'aria grazie alle 3 zone in mesh e alla zip centrale. tasche due pratiche tasche a rete sulla schiena per la nutrizione e i rifiuti!informazioni tecniche materiale ultraleggero per avere traspirabilitãƒâ‚¬ oltre al componente principale traspirante, sulle principali zone di sudorazione è stato inserito del mesh: sotto le ascelle e sui lati, oltre che in fondo alla schiena. cosa c'è di più confortevole che correre all'asciutto? dettagli dedicati al comfort dei praticanti di trail sviluppata per resistere agli sfregamenti, questa canotta ha anche dei rinforzi in silicone sulle spalle per tenere bene a posto lo zaino. il colletto protegge da eventuali irritazioni provocate dallo zaino e protegge dai colpi di sole! due tasche posizionate strategicamente le due tasche a rete di facile accesso e chiuse con una patta permettono di riporre i gel e i fazzolettini. e, secondo i nostri studi, la posizione sulla schiena è l'deale per l'equilibrio. vedi altri",0.0,1.0,si,no
maglia uomo agile m,98376728,df sport specialist,"Capo a maniche lunghe, ad asciugatura rapida e altamente traspirante per un utilizzo intensivo tutto l anno., Per avere comfort in una vasta gamma di temperature, la AGILE LS TEE da uomo unisce morbido poliestere ad asciugatura rapida ad inserti di aerazione strategicamente posizionati. Perfetta quando sudi molto, offre uno stile sobrio adatto a ogni tipo di attività. I dettagli riflettenti assicurano visibilità\ durante il running notturno., COMPOSIZIONE Insert 100% Polyester Body 100% Polyester, PESO (GRAMMI) 148, UTILIZZO, Runn

In [24]:
ds_dataset=spark.sql("select * from db_dettagli_prodotti_clean_5_csv")

In [25]:
#Applico al db impostando come condizione che il campo della descrizione non sia vuoto
ds_dataset.createOrReplaceTempView("ds_dataset")
dataset_cleaned = spark.sql("select *, regexp_replace(Descrizione, '[0-9]', ' ') as description_cleaned from ds_dataset where Descrizione is not null")
dataset_tokenized = regexTokenizer.transform(dataset_cleaned)
dataset_cleaned = remover.transform(dataset_tokenized)
dataset_filtered = dataset_cleaned.withColumn("filtered", filter_by_len_udf(col("cleaned")))
dataset_ngrams = ngrams2.transform(dataset_filtered)
dataset_ngrams = ngrams3.transform(dataset_ngrams)
dataset_ngrams_final = dataset_ngrams.filter("filtered is not Null").withColumn("ngrams", union_ngrams_udf(col("filtered"), col("ngrams_2"), col("ngrams_3")))

db_hash_dataset = hashing_tf.transform(dataset_ngrams_final)
db_idf_dataset = idf_model.transform(db_hash_dataset)
dataset_indexed = indexer_model.transform(db_idf_dataset)

In [26]:
db_predictions = nb_model.transform(dataset_indexed).select("*")
predictions_decoded = converter.transform(db_predictions)
display(predictions_decoded.select("*").limit(20))

Sito Id_prodotto Url Nome Tipo Tipo_specifico Brand Descrizione Genere Colore Prezzo Immagine Fattore_tecnico Tecnico Fattore_ambientale Ecologico 1 description_cleaned words cleaned filtered ngrams_2 ngrams_3 ngrams rawFeatures features rawPrediction probability prediction prediction_tecnico decathlon 8574003 https://www.decathlon.it/calze-basse-running-kiprun-id_8574003.html calze running basse kiprun arancioni altro calze kiprun le nostre equipe hanno sviluppato queste calze per fare in modo che il piede sia protetto dalle vesciche durante le uscite di running.perché delle calze sottili? aderenti al piede, con una lavorazione con filo anti-frizioni, queste calze da running molto corte conquisteranno i runner che cercano sensazioni!i vantaggi
limitazione irritazioni
fili specifici che limitano il rischio di vesciche sulla punta e sul tallone.
sostegno
griglia di misure fitta per una regolazione precisa e fascia elastica.
traspirazione
grazie al tessuto traspirante.informazioni tecniche
filo utilizzato sul tallone e sulla punta
sul tallone e sulla punta, il componente è poliammide ritorto con un secondo filo di poliammide rivestito in ptfe che permette di ridurre la comparsa di vesciche.
cucitura specifica sulla punta
sulla punta è presente una costruzione specifica che permette alle calze di seguire perfettamente l'area delle dita. unisex nero 6.39 img_8574003.jpg null no null no null le nostre equipe hanno sviluppato queste calze per fare in modo che il piede sia protetto dalle vesciche durante le uscite di running.perché delle calze sottili? aderenti al piede, con una lavorazione con filo anti-frizioni, queste calze da running molto corte conquisteranno i runner che cercano sensazioni!i vantaggi
limitazione irritazioni
fili specifici che limitano il rischio di vesciche sulla punta e sul tallone.
sostegno
griglia di misure fitta per una regolazione precisa e fascia elastica.
traspirazione
grazie al tessuto traspirante.informazioni tecniche
filo utilizzato sul tallone e sulla punta
sul tallone e sulla punta, il componente è poliammide ritorto con un secondo filo di poliammide rivestito in ptfe che permette di ridurre la comparsa di vesciche.
cucitura specifica sulla punta
sulla punta è presente una costruzione specifica che permette alle calze di seguire perfettamente l'area delle dita. List(le, nostre, equipe, hanno, sviluppato, queste, calze, per, fare, in, modo, che, il, piede, sia, protetto, dalle, vesciche, durante, le, uscite, di, running, perché, delle, calze, sottili, aderenti, al, piede, con, una, lavorazione, con, filo, anti, frizioni, queste, calze, da, running, molto, corte, conquisteranno, i, runner, che, cercano, sensazioni, i, vantaggi, limitazione, irritazioni, fili, specifici, che, limitano, il, rischio, di, vesciche, sulla, punta, e, sul, tallone, sostegno, griglia, di, misure, fitta, per, una, regolazione, precisa, e, fascia, elastica, traspirazione, grazie, al, tessuto, traspirante, informazioni, tecniche, filo, utilizzato, sul, tallone, e, sulla, punta, sul, tallone, e, sulla, punta, il, componente, è, poliammide, ritorto, con, un, secondo, filo, di, poliammide, rivestito, in, ptfe, che, permette, di, ridurre, la, comparsa, di, vesciche, cucitura, specifica, sulla, punta, sulla, punta, è, presente, una, costruzione, specifica, che, permette, alle, calze, di, seguire, perfettamente, l, area, delle, dita) List(equipe, sviluppato, calze, fare, modo, piede, protetto, vesciche, durante, uscite, running, calze, sottili, aderenti, piede, lavorazione, filo, anti, frizioni, calze, running, molto, corte, conquisteranno, runner, cercano, sensazioni, vantaggi, limitazione, irritazioni, fili, specifici, limitano, rischio, vesciche, punta, tallone, sostegno, griglia, misure, fitta, regolazione, precisa, fascia, elastica, traspirazione, grazie, tessuto, traspirante, informazioni, tecniche, filo, utilizzato, tallone, punta, tallone, punta, componente, poliammide, ritorto, secondo, filo, poliammide, rivestito, ptfe, perm

In [27]:
db_product_prediction = predictions_decoded.select ("id_prodotto", "prediction_tecnico")

In [28]:
db_product_prediction.createOrReplaceTempView("db_product_prediction")

In [29]:
%sql

create table pred as
(select id_prodotto as prod, prediction_tecnico from db_product_prediction)